# **<span style="font-family: 'Palatino Linotype', serif;">⚙️⚡Forma, função e ativação</span>**
----
*<span style="font-family: 'Angilla Tattoo'"> ""Moldaremos a forma, otimizaremos a função e executataremos a ativação da magia pato-digital para consquistar o Reino de Lumi! 🦆💻✨""</span>*

<div align="center">
    <img src = "Maga Forma, Função e ativação.jpg" alt = "Maga FFA" width = 300>
</div>

----
 **Objetivo:** Nesse notebook, implementei 3 as funções de ativação função liner, Tahn e Softmax, na rede neural feita em python em sala da aula. Realizo uma breve explicação sobre as funções, mostando suas equações e diferenças em relação a função sigmoidal, e por fim são feitos testes simples para comprovar seu funcionamento.
 
---

### 📐 A função linear

A função é definida como $$ f(x) = ax $$ porém, a função linear é constante, o que significa que o valor de sua derivada não depende de X. Nesse sentido, o valor do gradiente local no backpropagation será sempre o mesmo. Isso é um grande problema, pois não podemos melhorar a função de perda "loss", e a saída final da nossa rede neural será uma transformação linear dos dados de entrada. Isso faz com que essa função de entrada seja interessante em contextos de problemas simples, onde desejamos que os resultados sejam interpretáveis.

In [112]:
# código que cria a classe Valor para operações matemáticas feito em sala de aula 
import math

class Valor_linear:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor_linear):
            outro_valor = Valor_linear(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor_linear(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor_linear):
            outro_valor = Valor_linear(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor_linear(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor_linear(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado
    
    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor_linear(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    ### função criada para realizar uma operação linear para cte = 2
    def funcao_linear(self):
        """Realiza a operação: x * cte = 2)"""
        return self.data * 2
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

In [113]:
# código que cria um neurônio artificial, feito em sala
import random

class Neuronio_linear:
    def __init__(self, num_dados_entrada):
        self.vies = Valor_linear(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor_linear(random.uniform(-1, 1)))
            
    def __call__(self, x):
        
        assert len(x) == len(self.pesos)
        
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  
        ### utiliza a função linear
        dado_de_saida = soma.funcao_linear()
        
        return dado_de_saida       
    
    def parametros(self):
        return self.pesos + [self.vies]

In [114]:
dados_de_entrada = [2, 3]

meu_neuronio = Neuronio_linear(len(dados_de_entrada))

print(meu_neuronio(dados_de_entrada))

1.2029610541699678


In [115]:
### código que cria uma camada de neurônios, feito em sala de aula
class Camada_linear:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio_linear(num_dados_entrada)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

In [116]:
dados_de_entrada = [2, 3]
num_neuronios = 5

minha_camada = Camada_linear(num_neuronios, len(dados_de_entrada))

print(minha_camada(dados_de_entrada))

[-2.0096194300279353, -3.605286388172126, 5.542409510143863, 2.6428576450306225, 1.4059801547422417]


In [117]:
### classe que cria a rede neural MLP feito em sala de aula
class MLP_linear:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):
        
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada_linear(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x):
        for camada in self.camadas:
            x = camada(x)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params
    

In [118]:
dados_de_entrada = [2, 3]
num_neuronios_por_camada = [2, 4, 5]

minha_mlp = MLP_linear(len(dados_de_entrada), num_neuronios_por_camada)

resultado = minha_mlp(dados_de_entrada)

print(resultado)

[15.020432300626572, -19.252502717781372, 4.50863805303381, -0.7858626891939915, 8.386044633544987]


---- 

###  🔢 A função Tanh

A função Tanh, ou tangente hiperbólica, é uma versão escalonada da função sigmoide, dada por $$ Tahn(x) = \frac {e^x - e^{-x}}{e^x - e^{-x}} $$ A função Tanh é simétrica entre os valore 1 e -1, diferente do que ocorre na função sigmoide, que é simétrica entre 0 e 1. Essa simetria em relação a origem é interessante para a rede neural na medida que evita que os valores propagados para o próximo neuronio sejam sempre positivos. Como a função não é linear, podemos realizar o backpropagation.

In [119]:
# código que cria a classe Valor para operações matemáticas feito em sala de aula 

class Valor_tanh:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor_tanh):
            outro_valor = Valor_tanh(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor_tanh(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor_tanh):
            outro_valor = Valor_tanh(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor_tanh(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor_tanh(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado
    
    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor_tanh(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    ### função criada para realizar a operação Tanh
    def tanh(self):
        """Realiza a operação da tangente hiperbólica"""
        return self.exp() - (-self.exp())/ (self.exp() + 1) - (-self.exp())
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

In [120]:
# código que cria um neurônio artificial, feito em sala

class Neuronio_tanh:
    def __init__(self, num_dados_entrada):
        self.vies = Valor_tanh(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor_tanh(random.uniform(-1, 1)))
            
    def __call__(self, x):
        
        assert len(x) == len(self.pesos)
        
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  
        ### utiliza a Tanh
        dado_de_saida = soma.tanh()
        
        return dado_de_saida       
    
    def parametros(self):
        return self.pesos + [self.vies]

In [121]:
dados_de_entrada = [7, 6, 5, 3]

meu_neuronio = Neuronio_tanh(len(dados_de_entrada))

print(meu_neuronio(dados_de_entrada))

Valor(data=99931.095333284)


In [122]:
### código que cria uma camada de neurônios, feito em sala de aula
class Camada_tanh:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio_tanh(num_dados_entrada)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

In [123]:
dados_de_entrada = [7, 6, 5, 3]
num_neuronios = 4

minha_camada = Camada_linear(num_neuronios, len(dados_de_entrada))

print(minha_camada(dados_de_entrada))

[6.62561314364077, -8.920461469161356, 1.358341458988189, -12.95110806798985]


In [124]:
### classe que cria a rede neural MLP feito em sala de aula
class MLP_tanh:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):
        
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada_tanh(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x):
        for camada in self.camadas:
            x = camada(x)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params
    

In [125]:
dados_de_entrada = [7, 6, 5, 3]
num_neuronios_por_camada = [2, 4, 5]

minha_mlp = MLP_linear(len(dados_de_entrada), num_neuronios_por_camada)

resultado = minha_mlp(dados_de_entrada)

print(resultado)

[53.03678140995267, -11.96504168573132, -56.55977314905519, 0.6011882921724783, -31.452674881460602]


-----

### ➗ Função SoftMax

A função SoftMax é um tipo de função sigmóide, útil para lidar com problemas de classificação. Ela converte um vetor de K números reais e os normaliza em uma distribuição de probabilidade de K resultados possíveis. Isso quer dizer que, a entrada da função será um vetor contendo K números. Esses números serão normalizados, de forma que sua soma final seja equivalente a 1 e represente uma probabilidade. A função é dada por $$ \sigma(z)_i = \frac{e^{zi}}{\sum^z_{j = 1} e^{zj}}$$ pra i = 1. Ou seja, é aplicada a função exponencial para cada elemento de entrada Z e os valores são normalizados. 

In [126]:
# código que cria a classe Valor para operações matemáticas feito em sala de aula 
import numpy as np

class Valor_SoftMax:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor_SoftMax):
            outro_valor = Valor_SoftMax(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor_SoftMax(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor_SoftMax):
            outro_valor = Valor_SoftMax(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor_SoftMax(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor_SoftMax(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado
    
    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor_SoftMax(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    ### função criada para realizar a operação SoftMax
    def softmax(x):
        """Realiza a operação SoftMax sobre um vetor x"""
        e_x = np.exp(x - np.max(x))  # essa subtração evita que os valores estourem
        return e_x / e_x.sum() # aqui estamos normalizando os números
        
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

In [127]:
# código que cria um neurônio artificial, feito em sala
import random

class Neuronio_SoftMax:
    def __init__(self, num_dados_entrada):
        self.vies = Valor_SoftMax(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor_SoftMax(random.uniform(-1, 1)))
            
    def __call__(self, x):
        assert len(x) == len(self.pesos)
        soma = Valor_SoftMax(0)
        for xi, wi in zip(x, self.pesos):
            soma += xi * wi
        soma += self.vies
        return soma 
        
        ### utiliza o SoftMax
        soft_max(lista_valores)
      
        return resultado 
    
    def parametros(self):
        return self.pesos + [self.vies]

In [128]:
dados_de_entrada = [Valor_SoftMax(v) for v in [5, 12, 4, 6, 7, 13]]

meu_neuronio = Neuronio_SoftMax(len(dados_de_entrada))

print(meu_neuronio(dados_de_entrada))

Valor(data=-11.514448220667695)


In [129]:
### código que cria uma camada de neurônios, feito em sala de aula
class Camada_SoftMax:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio_SoftMax(num_dados_entrada)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

In [130]:
dados_de_entrada = [Valor_SoftMax(v) for v in [5, 12, 4, 6, 7, 13]]
num_neuronios = 7

minha_camada = Camada_SoftMax(num_neuronios, len(dados_de_entrada))

print(minha_camada(dados_de_entrada))

[Valor(data=14.989757378692323), Valor(data=5.014127832496978), Valor(data=-8.818856820670652), Valor(data=5.434187336946087), Valor(data=10.582332862822586), Valor(data=-0.6174172729290723), Valor(data=-2.8093976126639224)]


In [131]:
### classe que cria a rede neural MLP feito em sala de aula
class MLP_SoftMax:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):
        
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada_SoftMax(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x):
        for camada in self.camadas:
            x = camada(x)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params
    

In [132]:
dados_de_entrada = [Valor_SoftMax(v) for v in [5, 12, 4, 6, 7, 13]]
num_neuronios_por_camada = [5, 8, 3]

minha_mlp = MLP_SoftMax(len(dados_de_entrada), num_neuronios_por_camada)

resultado = minha_mlp(dados_de_entrada)

print(resultado)

[Valor(data=-39.701881028685925), Valor(data=1.2460714667891983), Valor(data=-22.926427433549417)]


-----

### 📊 Conclusão:

Nesse notebook, exploramos outras funções de ativação que não foram exploradas em sala, o que permitiu a compreensão do funcionamento matemático e computacional de diferentes funções. Pesquisar sobre as diferenças em relação a função sigmoide tornou possível entender que, para diferentes problemas, é necessário identificar qual a função de ativação ideal para se obter o resultado esperado.

----
### 📚 Referências:

CAPÍTULO 8 – Função de Ativação. Deep Learning Book, [s.d.]. Disponível em: <https://www.deeplearningbook.com.br/funcao-de-ativacao/>. Acesso em: 1 abr. 2025.

FUNÇÃO softmax. Wikipédia: a enciclopédia livre, [s.d.]. Disponível em: <https://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_softmax>. Acesso em: 1 abr. 2025.

TANGENTE hiperbólica. Wikipédia: a enciclopédia livre, [s.d.]. Disponível em: <https://pt.wikipedia.org/wiki/Tangente_hiperb%C3%B3lica>. Acesso em: 1 abr. 2025.

CHATGPT. OpenAI, [s.d.]. Disponível em: <https://chatgpt.com/share/67ec43f2-6d60-8005-96b7-0647b4325879>. Acesso em: 1 abr. 2025.

GEMINI. Google AI, [s.d.]. Disponível em: <https://gemini.google.com/app/b06bb6fbe78b91fc?hl=pt-BR>. Acesso em: 1 abr. 2025.